# OptCache Report Builder

Этот скрипт загружает данные оптимизации из CSV файлов и выводит топ 5 результатов по убыванию custom_fitness и profit.

In [ ]:
filter_rule = {
    'fields': {
        'profit': {
            'enabled_headmap': True,
            'enabled_report': False,
            'range': (0.01, float('inf')),
            'color_ranges': []
        },
        'trades_per_30_days': {
            'enabled_headmap': True,
            'enabled_report': False,
            'range': (0.5, float('inf')),
            'color_ranges': []
        },
        'profit_per_30_days_percent': {
            'enabled_headmap': False,
            'enabled_report': True,
            'range': (0.005, float('inf')),
            'color_ranges': [
                {'range': (-float('inf'), 0.010), 'color': 'pink'},
                {'range': (0.010, 0.020), 'color': 'moccasin'},
                {'range': (0.020, float('inf')), 'color': 'lightgreen'}
            ]
        },
        'profit_per_365_days_percent': {
            'enabled_headmap': True,
            'enabled_report': False,
            'range': (0.05, float('inf')),
            'color_ranges': [
                {'range': (-float('inf'), 0.050), 'color': 'pink'},
                {'range': (0.050, 0.070), 'color': 'moccasin'},
                {'range': (0.070, 0.100), 'color': '#C5E8B7'},
                {'range': (0.100, 0.150), 'color': '#ABE098'},
                {'range': (0.150, 0.200), 'color': '#83D475'},
                {'range': (0.200, 0.250), 'color': '#57C84D'},
                {'range': (0.250, float('inf')), 'color': '#2EB62C'}
            ]
        },
        'calmar': {
            'enabled_headmap': True,
            'enabled_report': False,
            'range': (1.5, float('inf')),
            'color_ranges': [
                {'range': (1.0, 1.5), 'color': 'moccasin'},
                {'range': (1.5, float('inf')), 'color': 'lightgreen'}
            ]
        },
        'win_rate': {
            'enabled_headmap': False,
            'enabled_report': False,
            'range': (0.0, float('inf')),
            'color_ranges': [
                {'range': (50.0, float('inf')), 'color': 'lightgreen'}
            ]
        },
        'custom_fitness': {
            'enabled_headmap': True,
            'enabled_report': False,
            'range': (0.75, float('inf')),
            'color_ranges': [
                {'range': (0.00, 0.75), 'color': 'pink'},
                {'range': (0.76, 0.90), 'color': 'moccasin'},
                {'range': (0.90, float('inf')), 'color': 'lightgreen'}
            ]
        },
        'reldrawdownpercnt_e': {
            'enabled_headmap': True,
            'enabled_report': False,
            'range': (0, 15),
            'color_ranges': [
                {'range': (0.00, 5.0),  'color': 'lightgreen'},
                {'range': (5.00, 15.0), 'color': 'moccasin'},
                {'range': (15.0, float('inf')), 'color': 'pink'}
            ]
        },
    },
    'max_runs_count': 100,
    'sort_by': ['sector', 'symbol', 'rank_score'],
    'sort_dir': [True, True, False]
}

# Scoring configuration for rank_score calculation
# Formula: rank_score = R² × norm(Calmar) × WeightedSum × 100
# Where WeightedSum = Σ(w_i × norm(m_i)) / Σw_i
scoring_config = {
    # Weights for weighted sum (any numbers, normalized automatically)
    'weights': {
        'sharpe_ratio': 3,
        'recovery_factor': 2,
        'win_rate': 2,
        'profit_factor': 1,
    }
}

In [209]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
import plotly.graph_objects as go
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.worksheet.table import Table, TableStyleInfo

In [ ]:
def LoadHeaderFromFileToDF(optcache_filename: str) -> pd.DataFrame:
    header_file = Path(f'{optcache_filename}.Header.csv')

    if not header_file.exists():
        print(f"File {header_file} do not exist")
        return pd.DataFrame()

    header_df = pd.read_csv(header_file, sep=';', encoding='utf-16')
    header_df['days'] = (pd.to_datetime(header_df['date_to']) - pd.to_datetime(header_df['date_from'])).dt.days
    header_df['filename'] = optcache_filename

    return header_df

def ClassifySymbol(symbol: str) -> str:
    """Classify financial symbol into category using local dictionaries and patterns"""
    
    symbol_upper = symbol.upper()
    
    # Known commodities
    commodities = {
        'GOLD', 'SILVER', 'COPPER', 'ALUMINIUM', 'ALUMINUM', 'PALLADIUM', 'PLATINUM',
        'BRENT', 'WTI', 'CRUDE', 'NGAS', 'NATGAS',
        'WHEAT', 'CORN', 'SOYBEAN', 'SUGAR', 'COFFEE', 'COCOA', 'COTTON',
        'XAUUSD', 'XAGUSD', 'XTIUSD'
    }
    
    # Known crypto prefixes
    crypto_prefixes = {'BTC', 'ETH', 'LTC', 'XRP', 'BCH', 'ADA', 'DOT', 'LINK', 'XLM', 'DOGE', 'MATIC', 'SOL', 'AVAX'}
    
    # Check exact match for commodities
    if symbol_upper in commodities:
        return 'Commodities'
    
    # Check crypto (BTCUSD, ETHUSD, etc)
    for prefix in crypto_prefixes:
        if symbol_upper.startswith(prefix):
            return 'Crypto'
    
    # Check indices (contain numbers like 50, 100, 500, 2000, etc)
    indices_keywords = ['SP', 'NAS', 'DOW', 'DAX', 'FTSE', 'NIKKEI', 'CHINA50', 'RUSSELL', 
                        'STOXX50', 'DXY', 'HK50', 'UK100', 'US500', 'USTEC']
    if symbol_upper in indices_keywords:
        return 'Indices'
    
    # Check Forex (6 chars, 3 currency codes)
    if len(symbol_upper) == 6 and symbol_upper.isalpha():
        currency_codes = {'USD', 'EUR', 'GBP', 'JPY', 'AUD', 'NZD', 'CAD', 'CHF', 'CNY', 'HKD', 'SGD'}
        first_part = symbol_upper[:3]
        second_part = symbol_upper[3:]
        if first_part in currency_codes and second_part in currency_codes:
            return 'Forex'
    
    # If nothing matched, return Other
    return 'Other'

def GroupSymbolsByCategory(symbols: list) -> dict:
    """Group symbols by their categories"""
    
    categories = {
        'Commodities': [],
        'Indices': [],
        'Crypto': [],
        'Stocks': [],
        'Forex': [],
        'Other': []
    }
    
    for symbol in symbols:
        category = ClassifySymbol(symbol)
        categories[category].append(symbol)
    
    # Remove empty categories and sort symbols within each category
    return {cat: sorted(syms) for cat, syms in categories.items() if syms}

def BuildHeaderReport(header_df: pd.DataFrame, data_df: pd.DataFrame) -> list[str]:
    if header_df.empty:
        return []

    row = header_df.iloc[0]

    # Format dates as YYYY-MM-DD
    date_from = pd.to_datetime(row["date_from"]).strftime("%Y-%m-%d")
    date_to = pd.to_datetime(row["date_to"]).strftime("%Y-%m-%d")

    symbols = list(map(str, pd.unique(data_df['symbol'])))
    grouped_symbols = GroupSymbolsByCategory(symbols)
    
    report = [
        f'## Backtest Optimization for {header_df["expert_name"].iloc[0]} / {len(symbols)} sym',
        f'<table>',
        f'    <tr>',
        f'        <td style="vertical-align: top"> ',
        f'            <b>Bot:</b> {row["expert_name"]} <br>',
        f'            <b>Interval:</b> [{date_from}; {date_to})<br>',
        f'            <b>Duration:</b> {row["days"]} days',
        f'        </td>',
        f'        <td style="vertical-align: top"> ',
        f'            <b>Server:</b> {row["server"]}<br>',
        f'            <b>Deposit:</b> {row["trade_deposit"]}<br>',
        f'            <b>Leverage:</b> 100',
        f'        </td>',
        f'    </tr>',
        f'    <tr>',
        f'        <td colspan=2>',
    ]

    # Add symbols grouped by category
    for category, syms in grouped_symbols.items():
        report.append(f'<b>{category}:</b> {", ".join(syms)}')
    
    report.append(f'        </td>')
    report.append(f'    </tr>')
    report.append(f'</table>')

    return report    

def CalculateRankScore(df: pd.DataFrame, config: dict) -> pd.Series:
    """
    Calculate rank_score for each row in DataFrame.
    
    Formula: rank_score = R² × norm(Calmar) × WeightedSum × 100
    
    Where:
    - R² = custom_fitness (already in [0, 1])
    - norm(Calmar) = Min-Max normalized calmar ratio
    - WeightedSum = Σ(w_i × norm(m_i)) / Σw_i
    
    Args:
        df: DataFrame with optimization results
        config: Scoring configuration with weights
    
    Returns:
        Series with rank_score values
    """
    if df.empty:
        return pd.Series(dtype=float)
    
    # Min-Max normalization helper
    def min_max_norm(series):
        min_val = series.min()
        max_val = series.max()
        if max_val == min_val:
            return pd.Series([1.0] * len(series), index=series.index)
        return (series - min_val) / (max_val - min_val)
    
    # Get R² (custom_fitness) - already in [0, 1]
    r2 = df['custom_fitness'].clip(0, 1)
    
    # Normalize Calmar
    calmar_norm = min_max_norm(df['calmar'])
    
    # Calculate weighted sum of other metrics
    weights = config.get('weights', {})
    total_weight = sum(weights.values())
    
    if total_weight == 0:
        weighted_sum = pd.Series([1.0] * len(df), index=df.index)
    else:
        weighted_sum = pd.Series([0.0] * len(df), index=df.index)
        for metric, weight in weights.items():
            if metric in df.columns:
                norm_metric = min_max_norm(df[metric])
                weighted_sum += weight * norm_metric
        weighted_sum = weighted_sum / total_weight
    
    # Calculate final rank_score
    return (r2 * calmar_norm * weighted_sum * 100).round(2)

def BuildReportDF(optcache_filename: str, filter: dict, filter_key: str = 'enabled_headmap') -> pd.DataFrame: 

    header_df = LoadHeaderFromFileToDF(optcache_filename)
    if header_df.empty:
        return pd.DataFrame()

    data_file = Path(f'{optcache_filename}.Data.csv')
    if not data_file.exists():
        print(f"File {data_file} do not exist")
        return pd.DataFrame()

    # add fields to data
    data_df = pd.read_csv(data_file, sep=';', encoding='utf-16')
    data_df['win_rate'] = data_df['profittrades'] / data_df['trades']
    data_df['profit_per_30_days'] = data_df['profit'] / header_df['days'].iloc[0] * 30
    data_df['profit_per_365_days'] = data_df['profit'] / header_df['days'].iloc[0] * 365
    data_df['profit_per_30_days_percent'] = data_df['profit'] / header_df['days'].iloc[0] * 30 / data_df['initial_deposit']
    data_df['profit_per_365_days_percent'] = data_df['profit'] / header_df['days'].iloc[0] * 365 / data_df['initial_deposit']
    data_df['trades_per_30_days'] = data_df['trades'] / header_df['days'].iloc[0] * 30
    
    # Calmar Ratio = Annual Return % / Max Drawdown %
    # Handle division by zero: if DD is 0, set calmar to a large value
    data_df['calmar'] = np.where(
        data_df['reldrawdownpercnt_e'] > 0,
        data_df['profit_per_365_days_percent'] * 100 / data_df['reldrawdownpercnt_e'],
        0  # Will be set to max after filtering
    )
   
    data_df['filename'] = optcache_filename
    data_df['symbol'] = header_df['symbol'].iloc[0]
    data_df['sector'] = header_df['sector'].iloc[0] if 'sector' in header_df.columns else ClassifySymbol(header_df['symbol'].iloc[0])
    data_df['expert_name'] = header_df['expert_name'].iloc[0]
    data_df['date_from'] = header_df['date_from'].iloc[0]
    data_df['date_to'] = header_df['date_to'].iloc[0]
    data_df['days'] = (pd.to_datetime(header_df['date_to'].iloc[0]) - pd.to_datetime(header_df['date_from'].iloc[0])).days
    data_df['months'] = (pd.to_datetime(header_df['date_to'].iloc[0]) - pd.to_datetime(header_df['date_from'].iloc[0])).days // 30
    data_df['period'] = header_df['period'].iloc[0]
    data_df['trade_deposit'] = header_df['trade_deposit'].iloc[0]
    data_df['trade_currency'] = header_df['trade_currency'].iloc[0]
    data_df['trade_leverage'] = header_df['trade_leverage'].iloc[0]
    data_df['server'] = header_df['server'].iloc[0]
    data_df['ticks_mode'] = header_df['ticks_mode'].iloc[0]
    data_df['rank_score'] = CalculateRankScore(data_df, scoring_config)

    data_df.fillna(0, inplace=True)

    # filter data
    report_data = data_df.copy()
    for column, rule in filter.get('fields', {}).items():
        if not rule.get(filter_key, True):
            continue
        if not rule.get('range'):
            continue
        min_val, max_val = rule['range']
        report_data = report_data[(report_data[column] >= min_val) & (report_data[column] <= max_val)]

    report_data = report_data.sort_values(filter_rule.get('sort_by', []), ascending=filter_rule.get("sort_dir", []))
    if 'max_runs_count' in filter:
        report_data = report_data.head(filter.get('max_runs_count'))

    return report_data

def CreateSetFile(optcache_filename: str, pass_number: int, output_dir: str = None, 
                  server: str = None, timestamp: str = None) -> str:
    """
    Create a .set file from optimization data for a specific pass.
    
    Args:
        optcache_filename: Path to the .opt file
        pass_number: Pass number to extract parameters from
        output_dir: Output directory for .set file (default: same as opt file)
        server: Server name for filename
        timestamp: Timestamp string for filename (YYYYMMDD_HHMMSS)
    
    Returns:
        Path to created .set file or empty string if failed
    """
    
    # Load header to get expert name and symbol
    header_df = LoadHeaderFromFileToDF(optcache_filename)
    if header_df.empty:
        print(f"Cannot load header from {optcache_filename}")
        return ""
    
    expert_name = header_df['expert_name'].iloc[0]
    symbol = header_df['symbol'].iloc[0]
    
    # Load data to find the specific pass
    data_file = Path(f'{optcache_filename}.Data.csv')
    if not data_file.exists():
        print(f"File {data_file} does not exist")
        return ""
    
    data_df = pd.read_csv(data_file, sep=';', encoding='utf-16')
    
    # Find the pass
    pass_data = data_df[data_df['Pass'] == pass_number]
    if pass_data.empty:
        print(f"Pass {pass_number} not found in {data_file}")
        return ""
    
    # Get parameter columns - all columns after 'avgconloosers'
    all_columns = list(data_df.columns)
    try:
        # Find the index of the last standard MetaTrader field
        last_standard_field_idx = all_columns.index('avgconloosers')
        # All columns after this are optimization parameters
        param_columns = all_columns[last_standard_field_idx + 1:]
    except ValueError:
        # If 'avgconloosers' not found, fall back to 'Inp_' prefix
        param_columns = [col for col in all_columns if col.startswith('Inp_')]
    
    # Filter out unnamed columns and empty column names
    param_columns = [col for col in param_columns if not col.startswith('Unnamed:') and col.strip() != '']
    
    if not param_columns:
        print(f"No input parameters found (columns after 'avgconloosers')")
        return ""
    
    # Create .set file content
    set_content = [
        "; saved automatically on generation",
        "; this file contains input parameters for testing/optimizing expert advisor",
        ";"
    ]
    
    # Add each parameter
    pass_row = pass_data.iloc[0]
    for param in param_columns:
        value = pass_row[param]
        
        # Skip NaN values
        if pd.isna(value):
            continue
        
        # Determine parameter type and format value
        if isinstance(value, (int, np.integer)):
            set_content.append(f"{param}={int(value)}")
        elif isinstance(value, (float, np.floating)):
            set_content.append(f"{param}={value}")
        else:
            set_content.append(f"{param}={value}")
    
    # Determine output path
    if output_dir is None:
        output_dir = Path(optcache_filename).parent
    else:
        output_dir = Path(output_dir)
    
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Create .set filename: {expert_name}_{server}_{timestamp}_{symbol}_{pass_number}.set
    set_filename = output_dir / f"{expert_name}_{server}_{timestamp}_{symbol}_{pass_number}.set"
    
    # Write .set file
    try:
        with open(set_filename, 'w', encoding='utf-8') as f:
            f.write('\n'.join(set_content))
        return str(set_filename)
    except Exception as e:
        print(f"Error writing .set file: {e}")
        return ""

In [ ]:
def BuildReportExcel(header_df: pd.DataFrame, data_df: pd.DataFrame, filter_rule: dict, 
                     all_symbols: list, output_filename: str, heatmap_df: pd.DataFrame = None,
                     timestamp: str = None) -> str:
    """Build Excel report with Heatmap sheet + SmartTable (auto-filter) and color coding
    
    Features:
    - Heatmap sheet with symbol overview (first sheet) - uses unfiltered data
    - Data sheet with SmartTable and filtering/sorting
    - Color coding for metrics (same as MD/HTML reports)
    - Frozen header row
    - Auto-width columns
    - Header with report info
    
    Args:
        header_df: Header DataFrame
        data_df: DataFrame with filtered results
        filter_rule: Filter rules dictionary
        all_symbols: List of all symbols
        output_filename: Output Excel filename
        heatmap_df: DataFrame with unfiltered results for heatmap (optional, uses data_df if None)
        timestamp: Timestamp string for set file names (YYYYMMDD_HHMMSS)
    
    Returns:
        Path to saved Excel file
    """
    
    # Use heatmap_df for heatmap if provided, otherwise use data_df
    heatmap_source_df = heatmap_df if heatmap_df is not None else data_df
    
    def get_fill_color(field_name, value):
        """Get fill color for a value based on filter_rule color_ranges"""
        rule = filter_rule.get('fields', {}).get(field_name, {})
        color_ranges = rule.get('color_ranges', [])
        for cr in color_ranges:
            if value >= cr['range'][0] and value < cr['range'][1]:
                color = cr['color']
                # Convert color names/hex to RGB for openpyxl
                color_map = {
                    'pink': 'FFB6C1',
                    'moccasin': 'FFE4B5',
                    'lightgreen': '90EE90',
                    '#C5E8B7': 'C5E8B7',
                    '#ABE098': 'ABE098',
                    '#83D475': '83D475',
                    '#57C84D': '57C84D',
                    '#2EB62C': '2EB62C',
                }
                return color_map.get(color, color.replace('#', '') if color.startswith('#') else None)
        return None
    
    # Calculate max calmar from heatmap source for gradient scaling
    max_calmar_for_gradient = 1.0
    if not heatmap_source_df.empty:
        # Get best calmar per symbol
        best_calmars = []
        for symbol in all_symbols:
            symbol_df = heatmap_source_df[heatmap_source_df['symbol'] == symbol]
            if not symbol_df.empty:
                best_calmars.append(symbol_df['calmar'].max())
        if best_calmars:
            max_calmar_for_gradient = max(max_calmar_for_gradient, max(best_calmars))
    
    def get_calmar_fill(calmar, count, filtered_count):
        """Get fill color for heatmap cell based on calmar value with dynamic gradient
        
        Colors:
        - Gray: No data at all (count=0)
        - Pink: Best Calmar <= 1
        - Yellow (moccasin): Calmar > 1, but no results after filtering (filtered_count=0)
        - Green gradient: Calmar > 1 AND has results after filtering
        """
        if count == 0:
            return 'D3D3D3'  # Gray - no data
        if calmar <= 1.0:
            return 'FFB6C1'  # Pink - Calmar <= 1
        if filtered_count == 0:
            return 'FFE4B5'  # Yellow (moccasin) - Calmar > 1 but filtered out
        
        # Gradient green for calmar > 1 with filtered results
        # Scale from 1 to max_calmar_for_gradient
        # Colors from light green to dark green
        green_colors = ['C5E8B7', 'ABE098', '83D475', '57C84D', '2EB62C']
        
        if max_calmar_for_gradient <= 1.0:
            return green_colors[-1]  # Best green if no spread
        
        # Normalize calmar to [0, 1] range where 1.0 -> 0, max_calmar -> 1
        t = (calmar - 1.0) / (max_calmar_for_gradient - 1.0)
        t = min(max(t, 0), 1)  # Clamp to [0, 1]
        
        # Map to color index
        idx = int(t * (len(green_colors) - 1))
        idx = min(idx, len(green_colors) - 1)
        
        return green_colors[idx]
    
    # Get header info
    hdr_row = header_df.iloc[0]
    date_from = pd.to_datetime(hdr_row["date_from"]).strftime("%Y-%m-%d")
    date_to = pd.to_datetime(hdr_row["date_to"]).strftime("%Y-%m-%d")
    
    # Create workbook
    wb = Workbook()
    
    # ==================== HEATMAP SHEET (First) ====================
    ws_heatmap = wb.active
    ws_heatmap.title = "Heatmap"
    
    # Collect symbol data for heatmap (using unfiltered data)
    symbols_data = []
    for symbol in sorted(all_symbols):
        symbol_df = heatmap_source_df[heatmap_source_df['symbol'] == symbol] if not heatmap_source_df.empty else pd.DataFrame()
        count = len(symbol_df)
        
        # Count filtered results for this symbol
        filtered_symbol_df = data_df[data_df['symbol'] == symbol] if not data_df.empty else pd.DataFrame()
        filtered_count = len(filtered_symbol_df)
        
        if count > 0:
            # Get best row by calmar (highest calmar = best)
            best_row = symbol_df.loc[symbol_df['calmar'].idxmax()]
            best_calmar = best_row['calmar']
            best_profit_pct = best_row['profit_per_365_days_percent'] * 100
            best_dd = best_row['reldrawdownpercnt_e']
            best_cc = best_row['custom_fitness']
            sector = best_row['sector'] if 'sector' in best_row else ClassifySymbol(symbol)
        else:
            best_calmar = 0
            best_profit_pct = 0
            best_dd = 0
            best_cc = 0
            sector = ClassifySymbol(symbol)
        
        symbols_data.append({
            'symbol': symbol,
            'count': count,
            'filtered_count': filtered_count,
            'calmar': best_calmar,
            'profit_pct': best_profit_pct,
            'dd': best_dd,
            'cc': best_cc,
            'sector': sector
        })
    
    # Sort by sector then symbol
    symbols_data.sort(key=lambda x: (x['sector'], x['symbol']))
    
    # Build filter description string
    filter_parts = []
    field_names = {
        'profit': 'Profit',
        'trades_per_30_days': 'Trades/mo',
        'profit_per_30_days_percent': '%/30d',
        'profit_per_365_days_percent': '%/yr',
        'calmar': 'Calmar',
        'win_rate': 'WR%',
        'custom_fitness': 'CC',
        'reldrawdownpercnt_e': 'DD%'
    }
    for field, rule in filter_rule.get('fields', {}).items():
        if rule.get('enabled_headmap', False):
            min_val, max_val = rule.get('range', (0, float('inf')))
            field_label = field_names.get(field, field)
            if max_val == float('inf'):
                filter_parts.append(f"{field_label}≥{min_val}")
            elif min_val == -float('inf') or min_val == 0:
                filter_parts.append(f"{field_label}≤{max_val}")
            else:
                filter_parts.append(f"{field_label}:[{min_val};{max_val}]")
    filter_str = "Filters: " + ", ".join(filter_parts) if filter_parts else "Filters: None"
    
    # Heatmap header
    ws_heatmap['A1'] = f"Heatmap: {hdr_row['expert_name']}"
    ws_heatmap['A1'].font = Font(bold=True, size=16)
    ws_heatmap.merge_cells('A1:F1')
    
    ws_heatmap['A2'] = f"{date_from} → {date_to} | {hdr_row['server']} | {len(all_symbols)} symbols | {len(data_df)} results"
    ws_heatmap['A2'].font = Font(size=11, color='666666')
    ws_heatmap.merge_cells('A2:F2')
    
    ws_heatmap['A3'] = filter_str
    ws_heatmap['A3'].font = Font(size=10, italic=True, color='888888')
    ws_heatmap.merge_cells('A3:F3')
    
    # Row 4: Color legend
    legend_items = [
        ('D3D3D3', 'No data'),
        ('FFB6C1', 'Calmar ≤ 1'),
        ('FFE4B5', 'Filtered out'),
        ('C5E8B7', 'Calmar > 1'),
        ('2EB62C', 'Best Calmar'),
    ]
    legend_thin_border = Border(
        left=Side(style='thin', color='AAAAAA'),
        right=Side(style='thin', color='AAAAAA'),
        top=Side(style='thin', color='AAAAAA'),
        bottom=Side(style='thin', color='AAAAAA')
    )
    for col_idx, (color, label) in enumerate(legend_items, 1):
        cell = ws_heatmap.cell(row=4, column=col_idx, value=label)
        cell.fill = PatternFill(start_color=color, end_color=color, fill_type='solid')
        cell.font = Font(size=9, bold=True)
        cell.alignment = Alignment(horizontal='center', vertical='center')
        cell.border = legend_thin_border
    ws_heatmap.row_dimensions[4].height = 20
    
    # Heatmap grid settings
    cols_per_row = 6
    cell_width = 18
    
    # Set column widths
    for col_idx in range(1, cols_per_row + 1):
        ws_heatmap.column_dimensions[chr(ord('A') + col_idx - 1)].width = cell_width
    
    # Build heatmap grid
    start_row = 6
    for idx, data in enumerate(symbols_data):
        row_num = start_row + (idx // cols_per_row) * 6  # 6 rows per heatmap row (symbol, CAGR, DD, Calmar, CC, gap)
        col_num = (idx % cols_per_row) + 1
        
        fill_color = get_calmar_fill(data['calmar'], data['count'], data['filtered_count'])
        cell_fill = PatternFill(start_color=fill_color, end_color=fill_color, fill_type='solid')
        center_align = Alignment(horizontal='center', vertical='center')
        thin_border = Border(
            left=Side(style='thin', color='AAAAAA'),
            right=Side(style='thin', color='AAAAAA'),
            top=Side(style='thin', color='AAAAAA'),
            bottom=Side(style='thin', color='AAAAAA')
        )
        
        # Row 1: Symbol name (bold) - show filtered count in parentheses
        cell1 = ws_heatmap.cell(row=row_num, column=col_num, value=f"{data['symbol']}({data['filtered_count']})")
        cell1.font = Font(bold=True, size=12)
        cell1.fill = cell_fill
        cell1.alignment = center_align
        cell1.border = thin_border
        
        # Row 2: CAGR (profit per year %)
        cell2 = ws_heatmap.cell(row=row_num + 1, column=col_num, value=f"CAGR: {data['profit_pct']:.1f}%")
        cell2.fill = cell_fill
        cell2.alignment = center_align
        cell2.border = thin_border
        cell2.font = Font(size=10)

        # Row 3: DD (drawdown %)
        cell3 = ws_heatmap.cell(row=row_num + 2, column=col_num, value=f"DD: {data['dd']:.1f}%")
        cell3.fill = cell_fill
        cell3.alignment = center_align
        cell3.border = thin_border
        cell3.font = Font(size=10)

        # Row 4: Calmar ratio
        cell4 = ws_heatmap.cell(row=row_num + 3, column=col_num, value=f"Calmar: {data['calmar']:.1f}")
        cell4.fill = cell_fill
        cell4.alignment = center_align
        cell4.border = thin_border
        cell4.font = Font(size=10)

        # Row 5: CC (custom criterion)
        cell5 = ws_heatmap.cell(row=row_num + 4, column=col_num, value=f"CC: {data['cc']:.2f}")
        cell5.fill = cell_fill
        cell5.alignment = center_align
        cell5.border = thin_border
        cell5.font = Font(size=10)
    
    # Set row heights for heatmap
    total_heatmap_rows = ((len(symbols_data) - 1) // cols_per_row + 1) * 6
    for row in range(start_row, start_row + total_heatmap_rows):
        ws_heatmap.row_dimensions[row].height = 18
    
    # ==================== DATA SHEET (Second) ====================
    ws_data = wb.create_sheet(title="Data")
    
    # Add header info (rows 1-3)
    ws_data['A1'] = f"Optimization Report: {hdr_row['expert_name']}"
    ws_data['A1'].font = Font(bold=True, size=14)
    ws_data.merge_cells('A1:O1')
    
    ws_data['A2'] = f"Interval: {date_from} → {date_to} | Server: {hdr_row['server']} | Deposit: {hdr_row['trade_deposit']} | Symbols: {len(all_symbols)} | Results: {len(data_df)}"
    ws_data['A2'].font = Font(size=10, color='666666')
    ws_data.merge_cells('A2:O2')
    
    # Row 3: Filters
    ws_data['A3'] = filter_str
    ws_data['A3'].font = Font(size=10, italic=True, color='888888')
    ws_data.merge_cells('A3:O3')
    
    # Empty row 4
    ws_data['A4'] = ""
    
    # Define columns
    columns = [
        {'key': '#', 'title': '#', 'width': 6},
        {'key': 'symbol', 'title': 'Symbol', 'width': 10},
        {'key': 'sector', 'title': 'Sector', 'width': 12},
        {'key': 'Pass', 'title': 'Pass', 'width': 8},
        {'key': 'profit', 'title': 'Profit', 'width': 12},
        {'key': 'profit_per_365_days_percent', 'title': '%/yr', 'width': 8, 'color_field': 'profit_per_365_days_percent', 'multiply': 100},
        {'key': 'reldrawdownpercnt_e', 'title': 'DD%', 'width': 8, 'color_field': 'reldrawdownpercnt_e'},
        {'key': 'calmar', 'title': 'Calmar', 'width': 8, 'color_field': 'calmar'},
        {'key': 'custom_fitness', 'title': 'CC', 'width': 8, 'color_field': 'custom_fitness'},
        {'key': 'trades_per_30_days', 'title': 'Pos/mo', 'width': 8},
        {'key': 'win_rate', 'title': 'WR%', 'width': 8, 'multiply': 100},
        {'key': 'profit_factor', 'title': 'PF', 'width': 8},
        {'key': 'recovery_factor', 'title': 'RF', 'width': 8},
        {'key': 'sharpe_ratio', 'title': 'Sharpe', 'width': 8},
        {'key': 'rank_score', 'title': 'Rank', 'width': 8},
        {'key': 'set_file', 'title': 'Set File', 'width': 40},
    ]
    
    # Header row styles
    header_fill = PatternFill(start_color='4A5568', end_color='4A5568', fill_type='solid')
    header_font = Font(bold=True, color='FFFFFF')
    header_alignment = Alignment(horizontal='center', vertical='center')
    thin_border = Border(
        left=Side(style='thin', color='CCCCCC'),
        right=Side(style='thin', color='CCCCCC'),
        top=Side(style='thin', color='CCCCCC'),
        bottom=Side(style='thin', color='CCCCCC')
    )
    
    # Write header row (row 5)
    header_row_num = 5
    for col_idx, col in enumerate(columns, 1):
        cell = ws_data.cell(row=header_row_num, column=col_idx, value=col['title'])
        cell.fill = header_fill
        cell.font = header_font
        cell.alignment = header_alignment
        cell.border = thin_border
        ws_data.column_dimensions[cell.column_letter].width = col['width']
    
    # Sort data
    sorted_df = data_df.sort_values(
        filter_rule.get('sort_by', []), 
        ascending=filter_rule.get("sort_dir", [])
    )
    
    # Write data rows
    data_start_row = header_row_num + 1
    for row_idx, (_, row_data) in enumerate(sorted_df.iterrows(), 1):
        excel_row = data_start_row + row_idx - 1
        
        # Generate set file name for this row: {expert_name}_{server}_{timestamp}_{symbol}_{pass}.set
        set_file_name = f"{row_data['expert_name']}_{row_data['server']}_{timestamp}_{row_data['symbol']}_{int(row_data['Pass'])}.set"
        
        for col_idx, col in enumerate(columns, 1):
            # Get value
            if col['key'] == '#':
                value = row_idx
            elif col['key'] == 'set_file':
                value = set_file_name
            else:
                value = row_data.get(col['key'], '')
                # Apply multiplier if specified
                if 'multiply' in col and isinstance(value, (int, float)):
                    value = value * col['multiply']
            
            # Round numeric values
            if isinstance(value, float):
                if col['key'] in ['profit']:
                    value = round(value, 0)
                elif col['key'] in ['profit_per_365_days_percent', 'win_rate']:
                    value = round(value, 1)
                else:
                    value = round(value, 2)
            
            cell = ws_data.cell(row=excel_row, column=col_idx, value=value)
            cell.border = thin_border
            cell.alignment = Alignment(horizontal='center' if col_idx > 3 else 'left')
            
            # Apply color coding
            if 'color_field' in col:
                original_value = row_data.get(col['color_field'], 0)
                fill_color = get_fill_color(col['color_field'], original_value)
                if fill_color:
                    cell.fill = PatternFill(start_color=fill_color, end_color=fill_color, fill_type='solid')
    
    # Calculate table range
    last_row = data_start_row + len(sorted_df) - 1
    last_col_letter = chr(ord('A') + len(columns) - 1)
    table_range = f"A{header_row_num}:{last_col_letter}{last_row}"
    
    # Create SmartTable
    table = Table(displayName="OptimizationResults", ref=table_range)
    
    # Add table style (medium blue style with banded rows)
    style = TableStyleInfo(
        name="TableStyleMedium2",
        showFirstColumn=False,
        showLastColumn=False,
        showRowStripes=True,
        showColumnStripes=False
    )
    table.tableStyleInfo = style
    ws_data.add_table(table)
    
    # Freeze panes (freeze header row)
    ws_data.freeze_panes = f'A{data_start_row}'
    
    # Save workbook
    wb.save(output_filename)
    
    return output_filename

## 2. Сортировка и вывод топ 5 результатов

In [ ]:
dir = Path('csv')
set_output_dir = Path('set_files')

full_df = pd.DataFrame()

opt_files = sorted(dir.glob("*.opt"))
if not opt_files:
    print(f"No .opt files found in {dir}")

# Collect all symbols from all .opt header files (before filtering)
all_symbols = []
for opt_file in opt_files:
    hdr = LoadHeaderFromFileToDF(opt_file)
    if not hdr.empty:
        all_symbols.append(hdr['symbol'].iloc[0])
all_symbols = sorted(set(all_symbols))

# Build unfiltered dataframe for heatmap (best calmar per symbol without filtering)
unfiltered_df = pd.DataFrame()
for opt_file in opt_files:
    opt_df = BuildReportDF(opt_file, {'fields': {}})  # No filters
    if not opt_df.empty:
        unfiltered_df = pd.concat([unfiltered_df, opt_df], ignore_index=True)

# Build dataframe with heatmap filtering (for heatmap display)
heatmap_filtered_df = pd.DataFrame()
for opt_file in opt_files:
    opt_df = BuildReportDF(opt_file, filter_rule, filter_key='enabled_headmap')
    if not opt_df.empty:
        heatmap_filtered_df = pd.concat([heatmap_filtered_df, opt_df], ignore_index=True)

# Build dataframe with report filtering (for Data table and .set files)
report_filtered_df = pd.DataFrame()
for opt_file in opt_files:
    opt_df = BuildReportDF(opt_file, filter_rule, filter_key='enabled_report')
    if not opt_df.empty:
        report_filtered_df = pd.concat([report_filtered_df, opt_df], ignore_index=True)

# load header from the first .opt file
header_df = LoadHeaderFromFileToDF(opt_files[0])

# Get report parameters for filename
expert_name = header_df['expert_name'].iloc[0] if not header_df.empty else 'Unknown'
server = header_df['server'].iloc[0] if not header_df.empty else 'Unknown'
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Build and save Excel report with SmartTable
# - heatmap uses heatmap_filtered_df for filtered_count calculation
# - data table uses report_filtered_df
excel_filename = f"{expert_name}_{server}_{timestamp}.xlsx"
BuildReportExcel(header_df, report_filtered_df, filter_rule, all_symbols, excel_filename, heatmap_df=unfiltered_df, timestamp=timestamp)
print(f"Excel report saved: {excel_filename}")

# Create .set files for all records in the report (filtered by enabled_report)
if not report_filtered_df.empty:
    set_output_dir.mkdir(parents=True, exist_ok=True)
    
    # Sort report_filtered_df the same way as in the report
    sorted_df = report_filtered_df.sort_values(filter_rule.get('sort_by', []), ascending=filter_rule.get("sort_dir", []))
    
    print(f"\nCreating .set files in {set_output_dir}/...")
    created_count = 0
    
    for idx, row in sorted_df.iterrows():
        opt_filename = row['filename']
        pass_num = int(row['Pass'])
        
        set_file = CreateSetFile(
            opt_filename, 
            pass_num, 
            output_dir=str(set_output_dir),
            server=server,
            timestamp=timestamp
        )
        if set_file:
            created_count += 1
    
    print(f"Created {created_count} .set files")

Excel report saved: HTFVWAP-MT5-Bot-2.00_Tickmill-Demo_20251213_005517.xlsx

Creating .set files in set_files/...
Created 667 .set files
Created 667 .set files
